# 第5章: 係り受け解析

日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

KNP使うときのコマンド

`juman < ai.ja.txt | knp -simple -anaphora > ai.ja.txt.parsed`

In [1]:
import os
from typing import List, Dict

In [2]:
input_path = os.path.join(os.getcwd(), '../data/ai.ja.txt.parsed')

In [3]:
# 読み込み
def get_raw(path: str) -> str:
    with open(path, mode='r') as f:
        return f.read()

In [4]:
# 行ごとに読み込み
def get_lines(path: str) -> List[str]:
    with open(path, mode='r') as f:
        s = [s.strip() for s in f.readlines()]
    return s

In [5]:
# 全文
raw = get_raw(input_path)

In [6]:
# 文リスト
sentences = raw.split('EOS\n')

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [7]:
# 形態素を表すクラス
class Morph():
    def __init__(self, morph_dict: dict):
        self.surface = morph_dict['surface'] # 表層形
        self.base = morph_dict['base'] # 基本形
        self.pos = morph_dict['pos'] # 品詞
        self.pos1 = morph_dict['pos1'] # 品詞細分類1

Morph({'surface': 'a', 'base': 'b', 'pos': 'c', 'pos1': 'd'})

In [8]:
# 入力形態素から始まる行を渡すと、表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）の辞書を返す
def get_surface_base_pos_pos1(text: str) -> dict:
    morp = text.split(' ')
    surface = morp[0]
    base = morp[2]
    pos = morp[3]
    pos1 = morp[5]
    return {
        'surface': surface,
        'base': base,
        'pos': pos,
        'pos1': pos1
    }

tes = 'a b c d e f'
expect = {
    'surface': 'a',
    'base': 'c',
    'pos': 'd',
    'pos1': 'f'
}
get_surface_base_pos_pos1(tes) == expect

True

In [9]:
# 1文を渡すと、形態素リストを返す
def get_sentence_morphs(sentence: str) -> List:
    # 改行で分ける
    # 長さ0の文字列は除外(末尾の改行によって空白になる要素)
    lines = [l for l in sentence.split('\n') if len(l) != 0]
    ret_list = []
    # 行ごとに処理
    for l in lines:
        # #, *, +で始まる行は形態素でない情報なのでパス
        if l[:1] == '#' or l[:1] == '*' or l[:1] == '+':
            continue
        # その他の（入力形態素から始まる）場合
        else:
            # 形態素を辞書型で取得
            morph_dict = get_surface_base_pos_pos1(l)
            # インスタンス作ってリストに入れる
            ret_list.append(Morph(morph_dict))
    return ret_list

tes = sentences[2]
''.join([m.surface for m in get_sentence_morphs(tes)])

'人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。'

In [10]:
# 出力
output = [get_sentence_morphs(s) for s in sentences]

In [11]:
# 表示
''.join([o.surface for o in output[2]])

'人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。'

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

In [12]:
# 文節を表すクラス
class Chunk():
    def __init__(self, chunk_dict: dict):
        self.morphs = chunk_dict['morphs']
        self.dst = chunk_dict['dst']
        self.srcs = chunk_dict['srcs']

In [13]:
import re
# 文節を渡すと、係り先と形態素リストを返す
def get_chunk_morphs(sentence: str) -> List:
    chunk = {}
    lines = sentence.split('\n')
    # 係り先
    chunk['dst'] = int(re.match(r'-*[0-9]+', lines[0]).group())
    # 形態素リスト
    chunk['morphs'] = get_sentence_morphs('\n'.join(lines[1:]))
    return chunk

In [14]:
# 1文を渡すと、文節リストを返す
def get_sentence_chunks(sentence: str) -> List:
    chunks_raw = sentence.split('\n* ')[1:] # 先頭は、#から始まる解析文の情報なので除外
    chunks = {}
    for i, c in enumerate(chunks_raw):
        chunks[i] = get_chunk_morphs(c)
        chunks[i]['srcs'] = []
    for i, c in chunks.items():
        if c['dst'] != -1:
            chunks[c['dst']]['srcs'].append(i)
    return [Chunk(c) for c in chunks.values()]

In [15]:
[[m.surface for m in c.morphs] for c in get_sentence_chunks(sentences[2])]

[['人工', '知能'],
 ['（', 'じんこう', 'ちのう', '、', '、'],
 ['AI'],
 ['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'],
 ['「', '『', '計算', '（）', '』', 'と'],
 ['いう'],
 ['概念', 'と'],
 ['『', 'コンピュータ', '（）', '』', 'と'],
 ['いう'],
 ['道具', 'を'],
 ['用いて'],
 ['『', '知能', '』', 'を'],
 ['研究', 'する'],
 ['計算', '機', '科学', '（）', 'の'],
 ['一', '分野', '」', 'を'],
 ['指す'],
 ['語', '。'],
 ['「', '言語', 'の'],
 ['理解', 'や'],
 ['推論', '、'],
 ['問題', '解決', 'など', 'の'],
 ['知的', '行動', 'を'],
 ['人間', 'に'],
 ['代わって'],
 ['コンピューター', 'に'],
 ['行わ', 'せる'],
 ['技術', '」', '、', 'または', '、'],
 ['「', '計算', '機'],
 ['（', 'コンピュータ', '）', 'に'],
 ['よる'],
 ['知的な'],
 ['情報', '処理', 'システム', 'の'],
 ['設計', 'や'],
 ['実現', 'に'],
 ['関する'],
 ['研究', '分野', '」', 'と', 'も'],
 ['さ', 'れる', '。']]

In [16]:
# 出力
output = [get_sentence_chunks(s) for s in sentences]

## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．